The main question is how to calculate permutation test with repeated measures (RM) anova using different clusters (per-defined groups of channels). Similar to the MNE function: https://martinos.org/mne/stable/generated/mne.stats.permutation_cluster_test.html

In [1]:
import os, sys
import pandas as pd
import numpy as np
import mne
from gc import collect as clear
from eelbrain import *

try:
    if "winsound" not in sys.modules:
        import winsound
    def makeSound(freq = 6000, # Hz
              duration = 3000): # millisecond
        winsound.Beep(freq, duration)
except ImportError:
    if "os" not in sys.modules:
        import winsound
    def makeSound():
        os.system('say -v Amir''s Task finished!')

C:\Users\Amir\Anaconda3\envs\mne\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


Importing sample data with pandas.

It's 60 files (~260kb per file). 3 groups, 10 couples, 2 repeated measures. 

To make it simple I used balanced design. My real data is unbalanced. 

1. The first two columns represents the channels' name
2. alpha/beta/gamma represents the PLV values for the pair of channels
3. id - couples' number
4. group - 1/2/3 the group condition. Between subjects. 
5. measure - m/f the first/second measure. Within subjects, my repeated measure. 

In [2]:
files = [x[0]+"/"+f for x in os.walk("data") for f in x[2] if
               f.endswith(".csv")] 

chnames = ['Fp1-0', 'Fp2-0', 'F7-0', 'F8-0', 'F3-0', 'F4-0', 'Fz-0', 'FT9-0', 'FT10-0', 'FC5-0', 'FC1-0',
 'FC2-0', 'FC6-0', 'T7-0', 'C3-0', 'Cz-0', 'C4-0', 'T8-0', 'TP9-0', 'CP5-0', 'CP1-0', 'CP2-0',
 'CP6-0', 'TP10-0',  'P7-0', 'P3-0', 'Pz-0', 'P4-0', 'P8-0', 'O1-0', 'O2-0', 'Fp1-1', 'Fp2-1',
 'F7-1', 'F8-1', 'F3-1', 'F4-1', 'Fz-1', 'FT9-1', 'FT10-1', 'FC5-1', 'FC1-1', 'FC2-1', 'FC6-1',
 'T7-1', 'C3-1', 'Cz-1', 'C4-1', 'T8-1', 'TP9-1', 'CP5-1', 'CP1-1', 'CP2-1',  'CP6-1',  'TP10-1',
 'P7-1',  'P3-1',  'Pz-1',  'P4-1',  'P8-1',  'O1-1', 'O2-1']

fs = []
for file in files:
    df = pd.read_csv(file, usecols=['channel_1', 'channel_2', "alpha"], header=0).assign(id = pd.to_numeric(os.path.basename(file)[1:4]), group = pd.to_numeric(os.path.basename(file)[1]), measure = os.path.basename(file)[5])
    df[["channel_1","channel_2"]] = df[["channel_1","channel_2"]].replace(list(range(62)), chnames)
    df[["group"]] = df[["group"]].replace([1, 2, 3], ["rc", "bf", "st"])
    df[["id"]] = df[["id"]].astype(float)
    df = np.expand_dims(df, axis = 0)
    fs.append(df)

fs = np.transpose(np.concatenate(fs, axis=0), (0, 2, 1))

print("my shape is:", fs.shape, "(file, columns(channel_1, channel_2, alpha, id, group, measure), plv values)")

print(fs[0])

my shape is: (60, 6, 3844) (file, columns(channel_1, channel_2, alpha, id, group, measure), plv values)
[['Fp1-0' 'Fp1-0' 'Fp1-0' ... 'O2-1' 'O2-1' 'O2-1']
 ['Fp1-0' 'Fp2-0' 'F7-0' ... 'P8-1' 'O1-1' 'O2-1']
 [0.0 0.31032640154038965 -0.2362711853195596 ... -0.042475196670743925
  0.7321402521589316 0.0]
 [130.0 130.0 130.0 ... 130.0 130.0 130.0]
 ['rc' 'rc' 'rc' ... 'rc' 'rc' 'rc']
 ['f' 'f' 'f' ... 'f' 'f' 'f']]


Converting channel names from numeric to actual names, changing group names to string, and changing id type to float.

The suffix -0 or -1 reflects if it is the first or second participant. 

**Starting to use *eelbrain**.

Creating `ds()` object from my pandas Data.Frame. For string columns I used `factor`, for numeric columns I used `NDVar`. 

Allowed connectivities between sensors

In [3]:
connectivity = pd.read_csv("connectivity.csv")
connectivity = connectivity[["ch1", "ch2"]]
connectivity = connectivity.values

In [7]:
connectivity.shape

(732, 2)

In [4]:
sensor_dim = Categorial("sensor", chnames, connectivity)

TypeError: connectivity array needs to be integer type, got object

In [ ]:
ds = Dataset()

for cl, ct in zip(fs.columns, fs.dtypes.values):
    if ct != "object":
        ds[cl] = NDVar(fs[cl].values, dims = (Case))
    else:
        ds[cl] = Factor(fs[cl].values)


In [ ]:
ds.shape

In [ ]:
#Cleaning memory
del files, fs, cl, chnames
clear()

Some basic info about my dataset

In [ ]:
print(ds.summary())
print(ds.head(5))
print(ds.tail(5))

In [ ]:
ds.info

If I understood correctly, I should use the `info` attribute to create my clusters.

So, let's say I have three clusters for the following channels: 
1. F4 and F3
2. T7, CP5, and P7
3. CP6, T8, and P8

How can I add and use these clusters? If I want to see if F4*-0* (in my first participant) is related to F3*-1* (in my second participant)? or T7*-0* is related to T7*-1*?

Regular RM anova for the data. While it's a bit redundant, I wanted to see that I can run it.  

In [ ]:
print(test.anova(ds["alpha"], ds["group"] * ds["measure"]))

Next, I moved to permutation RM anova with clusters. 
1. I understand that I have some issues with the way I build the NDVar for `id`. I didn't understand how to fix it. 
2. How should I specify the cluster?  

In [ ]:
print(testnd.anova(ds["alpha"], ds["group"] * ds["measure"] * ds["id"], match=False, samples=100, title="RM with permutation:"))